# Building an Agent around a Query Pipeline

In this cookbook we show you how to build an agent around a query pipeline.

Agents offer the ability to do complex, sequential reasoning on top of any query DAG that you have setup. Conceptually this is also one of the ways you can add a "loop" to the graph.

We show you two examples of agents you can implement:
- a full ReAct agent that can do tool picking
- a "simple" agent that adds a retry layer around a text-to-sql query engine.

## Setup Data

We use the chinook database as sample data. [Source](https://www.sqlitetutorial.net/sqlite-sample-database/).

In [ ]:
!curl "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O ./chinook.zip
!unzip ./chinook.zip

In [2]:
from llama_index import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

In [3]:
from llama_index.query_pipeline import QueryPipeline

### Setup Observability

We setup Arize Phoenix for observability.

In [4]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index

px.launch_app()
llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://0.0.0.0:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Setup Text-to-SQL Query Engine / Tool

Now we setup a simple text-to-SQL tool: given a query, translate text to SQL, execute against database, and get back a result. 

In [5]:
from llama_index.query_engine import NLSQLTableQueryEngine
from llama_index.tools.query_engine import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True,
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

## Setup ReAct Agent Pipeline

We now setup a ReAct pipeline for a single step using our Query Pipeline syntax. This is a multi-part process that does the following:
1. Takes in agent inputs
2. Calls ReAct prompt using LLM to generate next action/tool (or returns a response).
3. If tool/action is selected, call tool pipeline to execute tool + collect response.
4. If response is generated, get response.

Throughout this we'll use a variety of agent-specific query components. Unlike normal query pipelines, these are specifically designed for query pipelines that are used in a `QueryPipelineAgentWorker`:
- An `AgentInputComponent` that allows you to convert the agent inputs (Task, state dictionary) into a set of inputs for the query pipeline.
- An `AgentFnComponent`: a general processor that allows you to take in the current Task, state, as well as any arbitrary inputs, and returns an output. In this cookbook we define a function component to format the ReAct prompt. However, you can put this anywhere.
- An `CustomAgentComponent`: similar to `AgentFnComponent`, you can implement `_run_component` to define your own logic, with access to Task and state. It is more verbose but more flexible than `AgentFnComponent` (e.g. you can define init variables, and callbacks are in the base class).

Note that any function passed into `AgentFnComponent` and `AgentInputComponent` MUST include `task` and `state` as input variables, as these are inputs passed from the agent. 

Note that the output of an agentic query pipeline MUST be `Tuple[AgentChatResponse, bool]`. You'll see this below.

我们现在使用我们的查询管道语法为单个步骤设置了一个ReAct管道。这是一个多步骤的过程，包括以下内容：

接收代理输入
调用LLM的ReAct提示以生成下一个动作/工具（或返回响应）。
如果选择了工具/动作，调用工具管道执行工具并收集响应。
如果生成了响应，获取响应。
在整个过程中，我们将使用各种特定于代理的查询组件。与普通的查询管道不同，这些组件专门设计用于QueryPipelineAgentWorker中使用的查询管道：

一个AgentInputComponent，它允许你将代理输入（任务，状态字典）转换为查询管道的一组输入。
一个AgentFnComponent：一个通用处理器，允许你接收当前的任务、状态以及任何任意输入，并返回一个输出。在这个教程中，我们定义了一个函数组件来格式化ReAct提示。然而，你可以把它放在任何地方。
一个CustomAgentComponent：类似于AgentFnComponent，你可以实现_run_component来定义你自己的逻辑，可以访问任务和状态。它比AgentFnComponent更冗长但更灵活（例如，你可以定义初始化变量，回调函数在基类中）。
注意，传递给AgentFnComponent和AgentInputComponent的任何函数必须包括task和state作为输入变量，因为这些是从代理传递的输入。

注意，代理查询管道的输出必须是Tuple[AgentChatResponse, bool]。你将在下面看到这一点。

### Define Agent Input Component

Here we define the agent input component, called at the beginning of every agent step. Besides passing along the input, we also do initialization/state modification.

In [6]:
from llama_index.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)
from llama_index.agent import Task, AgentChatResponse
from llama_index.query_pipeline import (
    AgentInputComponent,
    AgentFnComponent,
    CustomAgentComponent,
    ToolRunnerComponent,
    QueryComponent,
)
from llama_index.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


## Agent Input Component
## This is the component that produces agent inputs to the rest of the components
## Can also put initialization logic here.
def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict[str, Any]:
    """Agent input function.

    Returns:
        A Dictionary of output keys and values. If you are specifying
        src_key when defining links between this component and other
        components, make sure the src_key matches the specified output_key.

    """
    # initialize current_reasoning
    if "current_reasoning" not in state:
        state["current_reasoning"] = []
    reasoning_step = ObservationReasoningStep(observation=task.input)
    state["current_reasoning"].append(reasoning_step)
    return {"input": task.input}


agent_input_component = AgentInputComponent(fn=agent_input_fn)

### Define Agent Prompt

Here we define the agent component that generates a ReAct prompt, and after the output is generated from the LLM, parses into a structured object.

In [7]:
# def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict[str, Any]:
#     """代理输入函数。

#     返回:
#         一个包含输出键和值的字典。如果你在定义这个组件与其他组件之间的链接时指定了src_key，
#         确保src_key与指定的output_key匹配。

#     """
#     # 如果state中没有"current_reasoning"，则初始化它
#     if "current_reasoning" not in state:
#         state["current_reasoning"] = []
#     # 创建一个基于当前任务输入的推理步骤
#     reasoning_step = ObservationReasoningStep(observation=task.input)
#     # 将新的推理步骤添加到state的"current_reasoning"列表中
#     state["current_reasoning"].append(reasoning_step)
#     # 返回一个包含任务输入的字典
#     return {"input": task.input}


In [8]:
from llama_index.agent.react.formatter import ReActChatFormatter
from llama_index.query_pipeline import InputComponent, Link
from llama_index.llms import ChatMessage
from llama_index.tools import BaseTool


## define prompt function
def react_prompt_fn(
    task: Task, state: Dict[str, Any], input: str, tools: List[BaseTool]
) -> List[ChatMessage]:
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    return chat_formatter.format(
        tools,
        chat_history=task.memory.get() + state["memory"].get_all(),
        current_reasoning=state["current_reasoning"],
    )


react_prompt_component = AgentFnComponent(
    fn=react_prompt_fn, partial_dict={"tools": [sql_tool]}
)

In [9]:
# def react_prompt_fn(
#     task: Task, state: Dict[str, Any], input: str, tools: List[BaseTool]
# ) -> List[ChatMessage]:
#     """
#     根据当前任务、状态、输入和工具生成响应的聊天消息列表。

#     参数:
#     task -- 当前的任务对象。
#     state -- 包含当前状态信息的字典。
#     input -- 用户的输入字符串。
#     tools -- 可用工具对象的列表。

#     返回:
#     生成的聊天消息列表。
#     """
#     # 将用户输入添加到推理过程中
#     chat_formatter = ReActChatFormatter()
#     # 使用chat_formatter工具来格式化聊天消息
#     return chat_formatter.format(
#         tools,
#         chat_history=task.memory.get() + state["memory"].get_all(),  # 获取任务记忆和状态记忆中的所有信息
#         current_reasoning=state["current_reasoning"],  # 获取当前的推理信息
#     )


### Define Agent Output Parser + Tool Pipeline

Once the LLM gives an output, we have a decision tree:
1. If an answer is given, then we're done. Process the output
2. If an action is given, we need to execute the specified tool with the specified args, and then process the output.

Tool calling can be done via the `ToolRunnerComponent` module. This is a standalone module that takes in a list of tools, and can be "executed" with the specified tool name (every tool has a name) and tool action.

We implement this overall module `OutputAgentComponent` that subclasses `CustomAgentComponent`.

Note: we also implement `sub_query_components` to pass through higher-level callback managers to the tool runner submodule.

In [10]:
class OutputAgentComponent(CustomAgentComponent):
    """
    OutputAgentComponent 类用于处理语言模型的输出，并根据输出的类型执行相应的操作。

    该组件根据决策树对输出进行处理：
    1. 如果输出是一个答案，则直接处理输出。
    2. 如果输出是一个动作，则需要使用 ToolRunnerComponent 模块执行指定的工具和参数，然后处理输出。

    ToolRunnerComponent 是一个独立的模块，它接受一个工具列表，并可以使用指定的工具名称和动作来“执行”。

    OutputAgentComponent 作为 CustomAgentComponent 的子类实现，它还实现了 `sub_query_components` 方法，
    以便将高级回调管理器传递给 ToolRunnerComponent 子模块。

    属性:
        tool_runner: ToolRunnerComponent 实例，用于执行工具动作。
        ... (其他属性根据实际情况添加)

    方法:
        process_output(self, output): 根据语言模型的输出处理结果。
        ... (其他方法根据实际情况添加)
    """


In [11]:
def finalize_fn(
    task: Task,
    state: Dict[str, Any],
    reasoning_step: Any,
    is_done: bool = False,
    tool_output: Optional[Any] = None,
) -> Tuple[AgentChatResponse, bool]:
    """
    完成函数。

    在这里，我们取最新的推理步骤和一个工具输出（如果提供的话），
    并返回代理输出（并决定代理是否完成）。

    该函数返回一个`AgentChatResponse`和`is_done`元组。并且是
    传递给`QueryPipelineAgentWorker`的任何查询管道的最后一个组件。
    这是任何查询管道预期的返回类型。
    """
    current_reasoning = state["current_reasoning"]  # 从状态字典中获取当前推理步骤列表
    current_reasoning.append(reasoning_step)  # 将最新的推


In [12]:
# def finalize_fn(
#     task: Task,
#     state: Dict[str, Any],
#     reasoning_step: Any,
#     is_done: bool = False,
#     tool_output: Optional[Any] = None,
# ) -> Tuple[AgentChatResponse, bool]:
#     """
#     完成函数。

#     在这里，我们取最新的推理步骤和一个工具输出（如果提供的话），
#     并返回代理输出（并决定代理是否完成）。

#     该函数返回一个`AgentChatResponse`和`is_done`元组。并且是
#     传递给`QueryPipelineAgentWorker`的任何查询管道的最后一个组件。
#     这是任何查询管道预期的返回类型。
#     """
#     current_reasoning = state["current_reasoning"]  # 从状态字典中获取当前推理步骤列表
#     current_reasoning.append(reasoning_step)  # 将最新的推理步骤添加到列表中
#     # 如果提供了tool_output，则将其添加到当前推理中
#     if tool_output is not None:
#         observation_step = ObservationReasoningStep(
#             observation=str(tool_output)
#         )  # 创建一个观察推理步骤，包含工具输出的字符串表示
#         current_reasoning.append(observation_step)  # 将观察推理步骤添加到推理列表中
#     # 如果最后一个推理步骤是响应推理步骤，则获取响应字符串
#     if isinstance(current_reasoning[-1], ResponseReasoningStep):
#         response_step = cast(ResponseReasoningStep, current_reasoning[-1])  # 类型转换以确保正确的类型
#         response_str = response_step.response  # 从响应推理步骤中获取响应字符串
#     else:
#         response_str = current_reasoning[-1].get_content()  # 否则，从最后一个推理步骤获取内容

#     # 如果is_done为真，则将聊天消息添加到记忆中
#     # 注意：'memory'是`state`中的保留关键字，但你也可以添加你自己的
#     if is_done:
#         state["memory"].put(
#             ChatMessage(content=task.input, role=MessageRole.USER)
#         )  # 将用户的输入作为聊天消息添加到记忆中
#         state["memory"].put(
#             ChatMessage(content=response_str, role=MessageRole.ASSISTANT)
#         )  # 将助手的响应作为聊天消息添加到记忆中

#     return AgentChatResponse(response=response_str), is_done  # 返回代理聊天响应和完成状态


In [13]:
from typing import Set, Optional
from llama_index.agent.react.output_parser import ReActOutputParser


## Agent Output Component
## Process reasoning step/tool outputs, and return agent response
def finalize_fn(
    task: Task,
    state: Dict[str, Any],
    reasoning_step: Any,
    is_done: bool = False,
    tool_output: Optional[Any] = None,
) -> Tuple[AgentChatResponse, bool]:
    """Finalize function.

    Here we take the latest reasoning step, and a tool output (if provided),
    and return the agent output (and decide if agent is done).

    This function returns an `AgentChatResponse` and `is_done` tuple. and
    is the last component of the query pipeline. This is the expected
    return type for any query pipeline passed to `QueryPipelineAgentWorker`.

    """
    current_reasoning = state["current_reasoning"]
    current_reasoning.append(reasoning_step)
    # if tool_output is not None, add to current reasoning
    if tool_output is not None:
        observation_step = ObservationReasoningStep(
            observation=str(tool_output)
        )
        current_reasoning.append(observation_step)
    if isinstance(current_reasoning[-1], ResponseReasoningStep):
        response_step = cast(ResponseReasoningStep, current_reasoning[-1])
        response_str = response_step.response
    else:
        response_str = current_reasoning[-1].get_content()

    # if is_done, add to memory
    # NOTE: memory is a reserved keyword in `state`, but you can add your own too
    if is_done:
        state["memory"].put(
            ChatMessage(content=task.input, role=MessageRole.USER)
        )
        state["memory"].put(
            ChatMessage(content=response_str, role=MessageRole.ASSISTANT)
        )

    return AgentChatResponse(response=response_str), is_done


class OutputAgentComponent(CustomAgentComponent):
    """Output agent component."""

    tool_runner_component: ToolRunnerComponent
    output_parser: ReActOutputParser

    def __init__(self, tools, **kwargs):
        tool_runner_component = ToolRunnerComponent(tools)
        super().__init__(
            tool_runner_component=tool_runner_component,
            output_parser=ReActOutputParser(),
            **kwargs
        )

    def _run_component(self, **kwargs: Any) -> Any:
        """Run component."""
        chat_response = kwargs["chat_response"]
        task = kwargs["task"]
        state = kwargs["state"]
        reasoning_step = self.output_parser.parse(
            chat_response.message.content
        )
        if reasoning_step.is_done:
            return {
                "output": finalize_fn(
                    task, state, reasoning_step, is_done=True
                )
            }
        else:
            tool_output = self.tool_runner_component.run_component(
                tool_name=reasoning_step.action,
                tool_input=reasoning_step.action_input,
            )
            return {
                "output": finalize_fn(
                    task,
                    state,
                    reasoning_step,
                    is_done=False,
                    tool_output=tool_output,
                )
            }

    @property
    def _input_keys(self) -> Set[str]:
        return {"chat_response"}

    @property
    def _optional_input_keys(self) -> Set[str]:
        return {"is_done", "tool_output"}

    @property
    def _output_keys(self) -> Set[str]:
        return {"output"}

    @property
    def sub_query_components(self) -> List[QueryComponent]:
        return [self.tool_runner_component]


react_output_component = OutputAgentComponent([sql_tool])

In [14]:
# class OutputAgentComponent(CustomAgentComponent):
#     """输出代理组件。"""

#     tool_runner_component: ToolRunnerComponent  # 工具运行组件实例
#     output_parser: ReActOutputParser  # 输出解析器实例

#     def __init__(self, tools, **kwargs):
#         # 初始化工具运行组件
#         tool_runner_component = ToolRunnerComponent(tools)
#         # 调用父类构造函数
#         super().__init__(
#             tool_runner_component=tool_runner_component,
#             output_parser=ReActOutputParser(),
#             **kwargs
#         )

#     def _run_component(self, **kwargs: Any) -> Any:
#         """运行组件。"""
#         # 从kwargs中获取聊天响应、任务和状态
#         chat_response = kwargs["chat_response"]
#         task = kwargs["task"]
#         state = kwargs["state"]
#         # 使用输出解析器解析聊天响应内容
#         reasoning_step = self.output_parser.parse(
#             chat_response.message.content
#         )
#         # 判断解析后的步骤是否完成
#         if reasoning_step.is_done:
#             # 如果完成，则调用finalize_fn函数并返回输出
#             return {
#                 "output": finalize_fn(
#                     task, state, reasoning_step, is_done=True
#                 )
#             }
#         else:
#             # 如果未完成，则运行工具运行组件
#             tool_output = self.tool_runner_component.run_component(
#                 tool_name=reasoning_step.action,
#                 tool_input=reasoning_step.action_input,
#             )
#             # 调用finalize_fn函数并返回输出
#             return {
#                 "output": finalize_fn(
#                     task,
#                     state,
#                     reasoning_step,
#                     is_done=False,
#                     tool_output=tool_output,
#                 )
#             }

#     @property
#     def _input_keys(self) -> Set[str]:
#         # 定义组件输入键的集合
#         return {"chat_response"}

#     @property
#     def _optional_input_keys(self) -> Set[str]:
#         # 定义组件可选输入键的集合
#         return {"is_done", "tool_output"}

#     @property
#     def _output_keys(self) -> Set[str]:
#         # 定义组件输出键的集合
#         return {"output"}

#     @property
#     def sub_query_components(self) -> List[QueryComponent]:
#         # 返回子查询组件列表
#         return [self.tool_runner_component]


# # 创建OutputAgentComponent实例，传入sql_tool作为工具
# react_output_component = OutputAgentComponent([sql_tool])


### Stitch together Agent Query Pipeline

We can now stitch together the top-level agent pipeline: agent_input -> react_prompt -> llm -> react_output.

The last component is the if-else component that calls sub-components.

In [15]:
# 我们现在可以将顶层代理管道拼接在一起：agent_input -> react_prompt -> llm -> react_output。
# 最后一个组件是调用子组件的if-else组件。


In [16]:
from llama_index.query_pipeline import QueryPipeline as QP
from llama_index.llms import OpenAI

qp = QP(
    modules={
        "agent_input": agent_input_component,
        "react_prompt": react_prompt_component,
        "llm": OpenAI(model="gpt-4-1106-preview"),
        "react_output": react_output_component,
    },
    verbose=True,
)
qp.add_chain(["agent_input", "react_prompt", "llm", "react_output"])

In [17]:
# 导入QueryPipeline类，简称为QP，用于构建查询管道
from llama_index.query_pipeline import QueryPipeline as QP
# 导入OpenAI类，用于与OpenAI的语言模型进行交互
from llama_index.llms import OpenAI

# 创建一个QueryPipeline实例，配置其模块和参数
qp = QP(
    modules={
        # "agent_input"模块负责处理用户输入
        "agent_input": agent_input_component,
        # "react_prompt"模块负责根据用户输入生成提示信息
        "react_prompt": react_prompt_component,
        # "llm"模块使用OpenAI的GPT-4模型进行语言理解和生成
        "llm": OpenAI(model="gpt-4-1106-preview"),
        # "react_output"模块负责处理语言模型的输出
        "react_output": react_output_component,
    },
    # verbose=True表示在执行过程中将打印详细信息
    verbose=True,
)
# 将处理链中的模块按顺序添加到查询管道中
qp.add_chain(["agent_input", "react_prompt", "llm", "react_output"])


### Visualize Query Pipeline

In [18]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("agent_dag.html")

agent_dag.html


### Setup Agent Worker around Text-to-SQL Query Pipeline

This is our way to setup an agent around a text-to-SQL Query Pipeline

In [19]:
from llama_index.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.callbacks import CallbackManager

agent_worker = QueryPipelineAgentWorker(qp)
agent = AgentRunner(agent_worker, callback_manager=CallbackManager([]))

In [20]:
agent_worker.agent_components

[AgentFnComponent(partial_dict={'tools': [<llama_index.tools.query_engine.QueryEngineTool object at 0x7f389afb70a0>]}, fn=<function react_prompt_fn at 0x7f389b7ea9d0>, async_fn=None),
 OutputAgentComponent(partial_dict={}, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f389afaf460>, tool_runner_component=ToolRunnerComponent(partial_dict={}, tool_dict={'sql_tool': <llama_index.tools.query_engine.QueryEngineTool object at 0x7f389afb70a0>}, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f389afaff70>), output_parser=<llama_index.agent.react.output_parser.ReActOutputParser object at 0x7f389afaf160>)]

In [21]:
from llama_index.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.callbacks import CallbackManager

agent_worker = QueryPipelineAgentWorker(qp)
agent = AgentRunner(agent_worker, callback_manager=CallbackManager([]))

In [22]:
agent_worker.agent_components

[AgentFnComponent(partial_dict={'tools': [<llama_index.tools.query_engine.QueryEngineTool object at 0x7f389afb70a0>]}, fn=<function react_prompt_fn at 0x7f389b7ea9d0>, async_fn=None),
 OutputAgentComponent(partial_dict={}, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f389afaf460>, tool_runner_component=ToolRunnerComponent(partial_dict={}, tool_dict={'sql_tool': <llama_index.tools.query_engine.QueryEngineTool object at 0x7f389afb70a0>}, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f389afaff70>), output_parser=<llama_index.agent.react.output_parser.ReActOutputParser object at 0x7f389afaf160>)]

### Run the Agent

Let's try the agent on some sample queries.

In [23]:
# start task
task = agent.create_task(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

In [24]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='d62c542b-6400-413a-a1b0-d013138959f8' input='What are some tracks from the artist AC/DC? Limit it to 3' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method ...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Too...



> Running module react_output with input: 
chat_response: assistant: Thought: I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {"input": "SELECT track_name FROM tracks WHERE artist_name = 'AC/DC' LIMIT 3"}



AttributeError: 'str' object has no attribute 'get_single_table_info'

In [ ]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='79a8d443-5707-4632-82b2-51fd253cd294' input='What are some tracks from the artist AC/DC? Limit it to 3' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method ...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Too...

> Running module react_output with input: 
chat_response: assistant: Thought: The user has repeated the question, but I have already provided the answer using the tool. I will re

In [ ]:
step_output.is_last

True

In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

The top 3 tracks by AC/DC are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


## Setup Simple Retry Agent Pipeline for Text-to-SQL 

Instead of the full ReAct pipeline that does tool picking, let's try a much simpler agent pipeline that only does text-to-SQL, with retry-logic.

We try a simple text-based "retry" prompt where given the user input and previous conversation history, can generate a modified query that outputs the right result.

### Define Core Modules

- agent input
- retry prompt
- output processor (including a validation prompt)

In [ ]:
from llama_index.llms import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-1106-preview")

In [ ]:
from llama_index.agent import Task, AgentChatResponse
from typing import Dict, Any
from llama_index.query_pipeline import AgentInputComponent, AgentFnComponent


def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict:
    """Agent input function."""
    # initialize current_reasoning
    if "convo_history" not in state:
        state["convo_history"] = []
        state["count"] = 0
    state["convo_history"].append(f"User: {task.input}")
    convo_history_str = "\n".join(state["convo_history"]) or "None"
    return {"input": task.input, "convo_history": convo_history_str}


agent_input_component = AgentInputComponent(fn=agent_input_fn)

In [ ]:
from llama_index.prompts import PromptTemplate

retry_prompt_str = """\
You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will convert the query to a SQL statement. If the agent triggers an error,
then that will be reflected in the current conversation history (see below).

If the conversation history is None, use the user input. If its not None,
generate a new SQL query that avoids the problems of the previous SQL query.

Input: {input}
Convo history (failed attempts): 
{convo_history}

New input: """
retry_prompt = PromptTemplate(retry_prompt_str)

In [ ]:
from llama_index.response import Response
from typing import Tuple

validate_prompt_str = """\
Given the user query, validate whether the inferred SQL query and response from executing the query is correct and answers the query.

Answer with YES or NO.

Query: {input}
Inferred SQL query: {sql_query}
SQL Response: {sql_response}

Result: """
validate_prompt = PromptTemplate(validate_prompt_str)

MAX_ITER = 3


def agent_output_fn(
    task: Task, state: Dict[str, Any], output: Response
) -> Tuple[AgentChatResponse, bool]:
    """Agent output component."""
    print(f"> Inferred SQL Query: {output.metadata['sql_query']}")
    print(f"> SQL Response: {str(output)}")
    state["convo_history"].append(
        f"Assistant (inferred SQL query): {output.metadata['sql_query']}"
    )
    state["convo_history"].append(f"Assistant (response): {str(output)}")

    # run a mini chain to get response
    validate_prompt_partial = validate_prompt.as_query_component(
        partial={
            "sql_query": output.metadata["sql_query"],
            "sql_response": str(output),
        }
    )
    qp = QP(chain=[validate_prompt_partial, llm])
    validate_output = qp.run(input=task.input)

    state["count"] += 1
    is_done = False
    if state["count"] >= MAX_ITER:
        is_done = True
    if "YES" in validate_output.message.content:
        is_done = True

    return AgentChatResponse(response=str(output)), is_done


agent_output_component = AgentFnComponent(fn=agent_output_fn)

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)

qp = QP(
    modules={
        "input": agent_input_component,
        "retry_prompt": retry_prompt,
        "llm": llm,
        "sql_query_engine": sql_query_engine,
        "output_component": agent_output_component,
    },
    verbose=True,
)
qp.add_link("input", "retry_prompt", src_key="input", dest_key="input")
qp.add_link(
    "input", "retry_prompt", src_key="convo_history", dest_key="convo_history"
)
qp.add_chain(["retry_prompt", "llm", "sql_query_engine", "output_component"])

### Visualize Query Pipeline

In [ ]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("agent_dag.html")

agent_dag.html


### Define Agent Worker

In [ ]:
from llama_index.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.callbacks import CallbackManager

agent_worker = QueryPipelineAgentWorker(qp)
agent = AgentRunner(agent_worker, callback_manager=CallbackManager([]))

In [ ]:
response = agent.chat(
    "How many albums did the artist who wrote 'Restless and Wild' release? (answer should be non-zero)?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='741c0d59-fa40-44a2-acab-cc4c36fdf0c7' input="How many albums did the artist who wrote 'Restless and Wild' release? (answer should be non-zero)?" memory=ChatMemoryBuffer(token_limit=3000, toke...

> Running module retry_prompt with input: 
input: How many albums did the artist who wrote 'Restless and Wild' release? (answer should be non-zero)?
convo_history: User: How many albums did the artist who wrote 'Restless and Wild' release? (answer should be non-zero)?

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will convert the query to a SQL statement. I...

> Running module sql_qu